In [1]:
import pandas
import re
import os
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import InputLayer, Masking, LSTM, TimeDistributed, Dense
from math import ceil
from keras.callbacks import Callback, ModelCheckpoint

Using TensorFlow backend.


# Maximal State LSTM Simulation

## Overview

The goal of this notebook is to use 2017 Ford F-150 Ecoboost chasis dynamometer data from the Argonne National Laboratory to simulate the speed of the vehicle given the current brake pedal and accelerator pedal pressures. The model tested in this notebook is a sequence-to-sequence LSTM. Note that the model is end-to-end, not an encoder-decoder model. To attempt to address issue with the previous model, in additional to the brake and accelerator pedal positions, the previous speed is included as input features. Hopefully the LSTM can store all other important information internally and implicitly. If not, we can employ teacher forcing and store even more state information (like current gear/ratio, engine load/RPM, etc.) externally to help the LSTM.

## Data

This data is from the Downloadable Dynamometer Database and was generated at the Advanced Mobility Technology Laboratory (AMTL) at Argonne National Laboratory under the funding and guidance of the U.S. Department of Energy (DOE).

If you read the previous `MinimalStateLSTM.ipynb` notebook, skip down a few lines after the data is collected.

First, load a list of TSV data files. These individual files represent separate tests done on the vehicle. Each have different goals and simulation techniques which you can read about in the `./Dynamometer/2017 Ford F150 Ecoboost Test Summary.xlsx` file. The data is recorded at the frequency of 10Hz.

In [2]:
csvs = list(filter(lambda file: re.match(r'^(?!61706006)\d{8} Test Data\.txt$', file) is not None, os.listdir('./DynamometerData')))

We load each TSV into a Pandas dataframe. Note: you may not have enough memory to do this all in one pass. If so, load each individual TSV and only keep the important columns identified below.

In [3]:
dfs = [pandas.read_csv('./DynamometerData/' + csv, sep='\t', header=0) for csv in csvs]

We can get an idea of which columns we have access to. We are looking for columns recording accelerator pedal position, break pedal position, and resulting speed.

In [4]:
list(dfs[0].columns)

['Time[s]_RawFacilities',
 'Dyno_Spd[mph]',
 'Dyno_TractiveForce[N]',
 'Dyno_LoadCell[N]',
 'Distance[mi]',
 'Dyno_Spd_Front[mph]',
 'Dyno_TractiveForce_Front[N]',
 'Dyno_LoadCell_Front[N]',
 'Dyno_Spd_Rear[mph]',
 'Dyno_LoadCell_Rear[N]',
 'Dyno_TractiveForce_Rear[N]',
 'DilAir_RH[%]',
 'Tailpipe_Press[inH2O]',
 'Cell_Temp[C]',
 'Cell_RH[%]',
 'Cell_Press[inHg]',
 'Tire_Front_Temp[C]',
 'Tire_Rear_Temp[C]',
 'Drive_Trace_Schedule[mph]',
 'Exhaust_Bag',
 'Engine_Oil_Dipstick_Temp[C]',
 'Radiator_Air_Outlet_Temp[C]',
 'Engine_Bay_Temp[C]',
 'Cabin_Temp[C]',
 'Cabin_Upper_Vent_Temp[C]',
 'Cabin_Lower_Vent_Temp[C]',
 'Solar_Array_Ind_Temp[C]',
 'Eng_FuelFlow_Direct2[gps]',
 '12VBatt_Volt_Hioki_U1[V]',
 '12VBatt_Curr_Hioki_I1[A]',
 '12VBatt_Power_Hioki_P1[W]',
 'Alternator_Curr_Hioki_I2[A]',
 'Alternator_Power_Hioki_P2[W]',
 '12VBatt_Curr_Hi_Hioki_I3[A]',
 '12VBatt_Power_Hi_Hioki_P3[W]',
 'Eng_FuelFlow_Direct[ccps]',
 'Eng_Fuel_Temp_Direct[C]',
 'Time[s]',
 'Trans_shift_inprogress_CAN[]',


Before we select our columns, we need to know the maximum sequence recorded. We will round to the nearest 100 to allow for flexible batch sizes.

In [5]:
max_length = (ceil(max([len(df) for df in dfs]) / 100)) * 100
max_length

61600

Now we can convert the data frames into input and target sets of sequences. We will make the suffix padding -1 as that is an invalid input value. We should not use 0 as that has meaning in the sequence.

In [6]:
# Padding with invalid value -1.
X = np.full([len(dfs), max_length, 10], -1.)
Y = np.full([len(dfs), max_length, 8], -1.)

for i, df in enumerate(dfs):
       
    # Current
    X[i,:len(df)-1,0] += df['Brake_pressure_applied_PCM[]'].values[1:] + 1
    X[i,:len(df)-1,1] += df['Pedal_accel_pos_CAN[per]'].values[1:] + 1
    
    # Previous
    X[i,:len(df)-1,2] += df['Dyno_Spd[mph]'].values[:-1] + 1
    X[i,:len(df)-1,3] += df['Eng_throttle_electronic_control_actual_PCM[deg]'].values[:-1] + 1
    X[i,:len(df)-1,4] += df['Eng_throttle_position_PCM[per]'].values[:-1] + 1
    X[i,:len(df)-1,5] += df['Trans_gear_engaged_CAN[]'].values[:-1] + 1
    X[i,:len(df)-1,6] += df['Eng_load_PCM[per]'].values[:-1] + 1
    X[i,:len(df)-1,7] += df['Eng_speed_PCM[rpm]'].values[:-1] + 1
    X[i,:len(df)-1,8] += df['Trans_gear_ratio_measured_TCM[]'].values[:-1] + 1
    X[i,:len(df)-1,9] += df['Trans_output_shaft_speed_raw_TCM[rpm]'].values[:-1] + 1
        
    # Outputs
    Y[i,:len(df)-1,0] += df['Dyno_Spd[mph]'].values[1:] + 1
    Y[i,:len(df)-1,1] += df['Eng_throttle_electronic_control_actual_PCM[deg]'].values[1:] + 1
    Y[i,:len(df)-1,2] += df['Eng_throttle_position_PCM[per]'].values[1:] + 1      
    Y[i,:len(df)-1,3] += df['Trans_gear_engaged_CAN[]'].values[1:] + 1
    Y[i,:len(df)-1,4] += df['Eng_load_PCM[per]'].values[1:] + 1
    Y[i,:len(df)-1,5] += df['Eng_speed_PCM[rpm]'].values[1:] + 1
    Y[i,:len(df)-1,6] += df['Trans_gear_ratio_measured_TCM[]'].values[1:] + 1
    Y[i,:len(df)-1,7] += df['Trans_output_shaft_speed_raw_TCM[rpm]'].values[1:] + 1
    

We can now delete the data frames to force release of some memory.

In [7]:
del dfs

Because the LSTM network is sensitive to magnitude, we need to scale our data. Since the sigmoid activation is used, we scale to the range $[0, 1]$. We store the minimums and maximums to inverse the transform after training and testing.

In [8]:
NEW_MIN = 0.25
NEW_MAX = 0.75
OLD_PAD_VAL = -1.
NEW_PAD_VAL = 0.

X_mins, X_maxs = [], []

for k in range(X.shape[2]):
    
    X_mins.append(X[:,:,k][X[:,:,k] != OLD_PAD_VAL].min())
    X_maxs.append(X[:,:,k][X[:,:,k] != OLD_PAD_VAL].max())
    
X_std = np.full(X.shape, NEW_PAD_VAL)

for i in range(X.shape[0]):    
    for k in range(X.shape[2]):
        
        indices = np.where(X[i,:,k] != OLD_PAD_VAL)        
        X_std[i,indices,k] += ((X[i,indices,k] - X_mins[k]) / (X_maxs[k] - X_mins[k])) * (NEW_MAX - NEW_MIN) + NEW_MIN - NEW_PAD_VAL

Y_mins, Y_maxs = [], []

for k in range(Y.shape[2]):
    
    Y_mins.append(Y[:,:,k][Y[:,:,k] != OLD_PAD_VAL].min())
    Y_maxs.append(Y[:,:,k][Y[:,:,k] != OLD_PAD_VAL].max())

Y_std = np.full(Y.shape, NEW_PAD_VAL)

for i in range(Y.shape[0]):
    for k in range(Y.shape[2]):
        
        indices = np.where(Y[i,:,k] != OLD_PAD_VAL)
        Y_std[i,indices,k] += ((Y[i,indices,k] - Y_mins[k]) / (Y_maxs[k] - Y_mins[k])) * (NEW_MAX - NEW_MIN) + NEW_MIN - NEW_PAD_VAL

## Model

We are now ready to start developing a model for our sequence. Since our sequences are one-to-one, I will not bother with an encoder-decoder architecture yet. This aslo shouldn't be necessary as only the past data is necessary to predict a single element of the sequence.

Since our sequences are over 60,000 elements long, we are going to use a trick to divide the sequences into 100-element subsequences and use the `stateful` parameter in our LSTM layer. The `stateful` option will store and pass along the state of the LSTM between batches. In other words, the terminating state of the $i$th sequence in the batch will be the initial state of the $i$th sequence in the following batch. 

This means we have to be very careful when defining our inputs. Typically, the `batch_input_shape` is of the form `(batch_size, time_steps, features)`. However, we are not batching on the first axis of our data (the individual sequences) but the length of the sequence. So, our `batch_input_shape` will follow the form `(n_samples, batch_length, features)`. We have 95 sequences, and we will save 10\% to use as test sequences, our number of samples for training is 86. And, as stated before, the length of the subsequences will be 100 elements.

The `return_sequences` option will override the typical LSTM behavior of just returning the last output as we want to use the entire sequence. The `stateful` option is discussed above.

The output layer is a `TimeDistributed` `Dense` layer. This will apply the `linear` activation to every element in the output sequence from the LSTM.

In [17]:
train_model = Sequential()

train_model.add(InputLayer(batch_input_shape=(85, 100, X_std.shape[2])))
train_model.add(LSTM(200, return_sequences=True, stateful=True))
train_model.add(TimeDistributed(Dense(Y_std.shape[2], activation='linear')))

train_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (85, 100, 200)            168800    
_________________________________________________________________
time_distributed_2 (TimeDist (85, 100, 8)              1608      
Total params: 170,408
Trainable params: 170,408
Non-trainable params: 0
_________________________________________________________________


`RMSprop` is recommended for LSTM sequences and `mean_squared_error` is appropriate for our numeric output.

In [18]:
train_model.compile(optimizer='Adam', loss='mse')

## Training

Because of out slightly unusual batching strategy, we need to implement a custom generator. We could extend a `keras.layers.Sequential` class, but our batching strategy is not too unusual so it is easy enough to implement a Python generator. The infinite loop allows for variable length epochs.

In [19]:
def batch_generator(X, y, batch_size):
    while True:
        for i in range(0, X.shape[1], batch_size):        
            yield (X[:,i:i+batch_size,:], y[:,i:i+batch_size,:])

Because our batch size is fixed, it is not possible to do simultaneous training and validation (unless we halved our data). So, here we manually shuffle and split our data using a 90-10 split.

In [20]:
SPLIT = X_std.shape[0] - X_std.shape[0] // 10

indices = np.arange(0, X_std.shape[0])
np.random.shuffle(indices)

X_shuffled = X_std[indices,:,:]
Y_shuffled = Y_std[indices,:,:]

X_train, X_test = X_shuffled[:SPLIT,:,:], X_shuffled[SPLIT:,:,:]
Y_train, Y_test = Y_shuffled[:SPLIT,:,:], Y_shuffled[SPLIT:,:,:]

It is very important to realize that Keras does not reset the state in a stateful LSTM after each epoch. However, we do *not* want the state to carry over between epochs. Therefore, we manually reset after each epoch using a custom callback.

In [21]:
class ResetStates(Callback):
  
  def on_epoch_end(self, epoch, logs=None):
    self.model.reset_states()

In [22]:
reset_states = ResetStates()

Because we are in a notebook and we don't want to lose our model upon a disconnect, we will use the `ModelCheckpoint` callback to save the best model seen so far.

In [23]:
model_checkpoint = ModelCheckpoint('./Models/MaximalStateLSTM/E{epoch:03d}L{loss:06.4E}.hdf5', save_weights_only=True)

Now we are ready to train. We will run for 200 epochs. Note that `steps_per_epoch` is required because our generator has no length. This is the max sequence length divided by the batch length.

In [ ]:
train_model.fit_generator(batch_generator(X_train, Y_train, 100), epochs=200, steps_per_epoch=X.shape[1] / 100, callbacks=[reset_states, model_checkpoint])

Epoch 1/200
616/616 [==============================] - 129s 209ms/step - loss: 0.0013
Epoch 2/200
616/616 [==============================] - 129s 209ms/step - loss: 1.1809e-04
Epoch 3/200
616/616 [==============================] - 129s 209ms/step - loss: 7.7743e-05
Epoch 4/200
616/616 [==============================] - 129s 209ms/step - loss: 5.9385e-05
Epoch 5/200
616/616 [==============================] - 129s 209ms/step - loss: 5.3732e-05
Epoch 6/200
616/616 [==============================] - 129s 210ms/step - loss: 4.3678e-05
Epoch 7/200
616/616 [==============================] - 129s 209ms/step - loss: 4.6758e-05
Epoch 8/200
616/616 [==============================] - 129s 209ms/step - loss: 3.1416e-05
Epoch 9/200
616/616 [==============================] - 128s 208ms/step - loss: 3.0694e-05
Epoch 10/200
616/616 [==============================] - 128s 208ms/step - loss: 2.8129e-05
Epoch 11/200
616/616 [==============================] - 128s 208ms/step - loss: 3.4737e-05
Epoch 12/200